In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [18]:
url = "https://www.eazydiner.com/restaurants?city=hyderabad&location=hyderabad"

In [19]:
#Headers for request
Headers = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36 Edg/142.0.0.0','Accept-Language':'en-US ,en;q=0.5'})

In [20]:
#HTTP Request
webpage = requests.get(url, headers=Headers)

In [21]:
webpage

<Response [200]>

In [22]:
type(webpage.content)

bytes

In [25]:
#Soup Object cotaiing all data 
soup = BeautifulSoup(webpage.content,"html.parser")

In [34]:
#Fetch links as list of tag objects
links = soup.find_all("a",attrs={'class':'ellipsis listing_res_name__uVIN8'})

In [35]:
links

[<a class="ellipsis listing_res_name__uVIN8" href="https://www.eazydiner.com/hyderabad/sixty-three-degrees-gachibowli-hyderabad-692677?deal_types[]=postpaid">63 Degree Modern Regional Buffet</a>,
 <a class="ellipsis listing_res_name__uVIN8" href="https://www.eazydiner.com/hyderabad/golconda-pavilion-itc-kohenur-hyderabad-657734?deal_types[]=postpaid">Golconda Pavilion</a>,
 <a class="ellipsis listing_res_name__uVIN8" href="https://www.eazydiner.com/hyderabad/exotica-banjara-hills-hyderabad-648799?deal_types[]=postpaid">Exotica</a>,
 <a class="ellipsis listing_res_name__uVIN8" href="https://www.eazydiner.com/hyderabad/forefathers-road-no-45-jubilee-hills-hyderabad-696918?deal_types[]=postpaid">Forefathers</a>,
 <a class="ellipsis listing_res_name__uVIN8" href="https://www.eazydiner.com/hyderabad/the-ofen-banjara-hills-hyderabad-648800?deal_types[]=postpaid">The Ofen</a>,
 <a class="ellipsis listing_res_name__uVIN8" href="https://www.eazydiner.com/hyderabad/one8-commune-hitech-city-hyder

In [40]:
link = links[0].get('href')

In [44]:
product_link = " " + link 

In [45]:
product_link

' https://www.eazydiner.com/hyderabad/sixty-three-degrees-gachibowli-hyderabad-692677?deal_types[]=postpaid'

In [46]:
new_webpage = requests.get(product_link, headers=Headers)

In [47]:
new_webpage

<Response [200]>

In [48]:
new_soup = BeautifulSoup(new_webpage.content,"html.parser")

In [61]:
# ==============================================================
# EazyDiner Hyderabad Scraper (Next.js JSON Endpoint)
# ==============================================================
# Requirements:
# pip install requests pandas
# ==============================================================

import requests
import pandas as pd

# Actual data endpoint
url = "https://www.eazydiner.com/_next/data/9Iz0uKJzU8-Mwrfvo8KeI/en/hyderabad.json"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*"
}

# Fetch JSON data
response = requests.get(url, headers=headers)
if response.status_code == 200:
    data = response.json()
else:
    print(f"⚠️ Failed to fetch data. Status code: {response.status_code}")
    exit()

# ==============================================================
# Extract restaurants from the JSON structure
# ==============================================================

# The JSON hierarchy can vary; let's locate where restaurant info is stored
page_props = data.get("pageProps", {})
restaurant_data = (
    page_props.get("data", {}).get("restaurant_data")
    or page_props.get("restaurants")
    or []
)

# Parse results
records = []
for rest in restaurant_data:
    info = rest.get("info", {})
    records.append({
        "Restaurant Name": info.get("name"),
        "Cuisine": ", ".join([c.get("name") for c in info.get("cuisine", [])]) if info.get("cuisine") else None,
        "Location": info.get("locality", {}).get("name"),
        "Cost for Two": info.get("costText", {}).get("text"),
        "Rating": info.get("rating", {}).get("aggregate_rating"),
        "Votes": info.get("rating", {}).get("votes"),
        "URL": info.get("url")
    })

# Convert to DataFrame
df = pd.DataFrame(records)
print(df.head())
print(f"✅ Extracted {len(df)} restaurants.")

# Save to CSV
df.to_csv("eazydiner_hyderabad.csv", index=False, encoding="utf-8")
print("✅ Data saved to eazydiner_hyderabad.csv")


Empty DataFrame
Columns: []
Index: []
✅ Extracted 0 restaurants.
✅ Data saved to eazydiner_hyderabad.csv


In [62]:
import requests
import json

url = "https://www.eazydiner.com/_next/data/9Iz0uKJzU8-Mwrfvo8KeI/en/hyderabad.json"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

r = requests.get(url, headers=headers)
data = r.json()

# Pretty print the structure (first 2 levels only)
def explore_json(d, indent=0):
    if isinstance(d, dict):
        for k, v in d.items():
            print("  " * indent + f"- {k}: {type(v).__name__}")
            if indent < 2:
                explore_json(v, indent + 1)
    elif isinstance(d, list) and len(d) > 0:
        print("  " * indent + f"[0]: {type(d[0]).__name__}")
        if indent < 2:
            explore_json(d[0], indent + 1)

explore_json(data)


- pageProps: dict
  - data: dict
    - responseInfo: dict
    - data: dict
    - meta: dict
    - customer_bookings: list
  - userLocation: dict
    - name: str
    - code: str
    - city_code: str
    - city_name: str
    - lat: float
    - long: float
    - city_id: int
    - country_id: int
    - group_id: int
  - token: str
  - isDesktop: bool
  - pageName: str
  - isCityPage: int
  - metaData: dict
    - title: str
    - description: str
    - jsonSchema: list
  - footerSeo: dict
    - responseInfo: dict
    - data: dict
  - cityData: dict
    - responseInfo: dict
    - data: dict
    - meta_title: str
    - meta_description: str
  - userDetails: NoneType
  - domain: str
  - pageView: str
- __N_SSP: bool


In [63]:
# ==============================================================
# EazyDiner Hyderabad Scraper (Confirmed JSON Path)
# ==============================================================
# Requirements:
# pip install requests pandas
# ==============================================================

import requests
import pandas as pd

# Actual JSON endpoint (you found it!)
url = "https://www.eazydiner.com/_next/data/9Iz0uKJzU8-Mwrfvo8KeI/en/hyderabad.json"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*"
}

response = requests.get(url, headers=headers)
if response.status_code != 200:
    print(f"⚠️ Failed to fetch data, status code: {response.status_code}")
    exit()

json_data = response.json()

# Navigate to the correct nested path
restaurants = json_data.get("pageProps", {}).get("data", {}).get("data", [])

if not restaurants:
    print("⚠️ No restaurants found — JSON path may differ slightly.")
else:
    print(f"✅ Found {len(restaurants)} restaurants in JSON.")

# Parse restaurant details
records = []
for r in restaurants:
    name = r.get("name")
    location = r.get("locality", {}).get("name") if isinstance(r.get("locality"), dict) else None
    cuisines = ", ".join([c.get("name") for c in r.get("cuisine", [])]) if r.get("cuisine") else None
    rating = r.get("rating", {}).get("aggregate_rating") if isinstance(r.get("rating"), dict) else None
    cost = r.get("costText", {}).get("text") if isinstance(r.get("costText"), dict) else None
    votes = r.get("rating", {}).get("votes") if isinstance(r.get("rating"), dict) else None
    url_rest = r.get("url")

    records.append({
        "Restaurant Name": name,
        "Cuisine": cuisines,
        "Location": location,
        "Cost for Two": cost,
        "Rating": rating,
        "Votes": votes,
        "URL": url_rest
    })

# Convert to DataFrame and save
df = pd.DataFrame(records)
print(df.head())
print(f"✅ Extracted {len(df)} restaurants.")
df.to_csv("eazydiner_hyderabad.csv", index=False, encoding="utf-8")
print("✅ Data saved to eazydiner_hyderabad.csv")


✅ Found 4 restaurants in JSON.


AttributeError: 'str' object has no attribute 'get'

In [64]:
import requests, json

url = "https://www.eazydiner.com/_next/data/9Iz0uKJzU8-Mwrfvo8KeI/en/hyderabad.json"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

data = requests.get(url, headers=headers).json()

restaurants = data["pageProps"]["data"]["data"]

print("Total items:", len(restaurants))
print("Type of first item:", type(restaurants[0]))
print(json.dumps(restaurants[0], indent=2))



Total items: 4


KeyError: 0

In [65]:
import requests
import json

url = "https://www.eazydiner.com/_next/data/9Iz0uKJzU8-Mwrfvo8KeI/en/hyderabad.json"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

data = requests.get(url, headers=headers).json()

level2 = data["pageProps"]["data"]["data"]

# print all keys and types under this dictionary
for k, v in level2.items():
    print(f"{k} → {type(v).__name__}")

# optional: print a preview of one key’s value (the one that looks like it might contain restaurants)
print("\nSample preview of a nested key (choose one that looks relevant):")
first_key = list(level2.keys())[0]
print(f"First key: {first_key}")
print(json.dumps(level2[first_key], indent=2)[:2000])


viewTitle → str
view → str
banners → list
buckets → dict

Sample preview of a nested key (choose one that looks relevant):
First key: viewTitle
"cityhome"


In [66]:
import requests, json

url = "https://www.eazydiner.com/_next/data/9Iz0uKJzU8-Mwrfvo8KeI/en/hyderabad.json"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
data = requests.get(url, headers=headers).json()

buckets = data["pageProps"]["data"]["data"]["buckets"]

# Print all bucket keys
print("Bucket keys:")
for k in buckets.keys():
    print("-", k)

# Show preview of the first bucket (structure of restaurant data)
first_key = list(buckets.keys())[0]
print(f"\nPreview for first bucket: {first_key}")
print(json.dumps(buckets[first_key], indent=2)[:2000])


Bucket keys:
- main_meal_bucket
- box-small-new
- popular_restaurants
- locations
- cuisines
- trends
- partners

Preview for first bucket: main_meal_bucket
{
  "title": "",
  "sub_title": "",
  "items": []
}


In [67]:
import requests, json

url = "https://www.eazydiner.com/_next/data/9Iz0uKJzU8-Mwrfvo8KeI/en/hyderabad.json"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
data = requests.get(url, headers=headers).json()

popular = data["pageProps"]["data"]["data"]["buckets"]["popular_restaurants"]

print("Keys in popular_restaurants bucket:", popular.keys())

# Print preview of the items list
if "items" in popular:
    print(f"\nTotal items: {len(popular['items'])}")
    if len(popular["items"]) > 0:
        print(json.dumps(popular["items"][0], indent=2)[:2000])
    else:
        print("⚠️ No items found inside 'popular_restaurants'.")
else:
    print("⚠️ No 'items' key inside popular_restaurants.")


Keys in popular_restaurants bucket: dict_keys(['title', 'sub_title', 'sub_title_link', 'items'])

Total items: 6
{
  "id": 684079,
  "action_url": "hyderabad/fuel-cafe-banjara-hills-hyderabad-684079",
  "image": "https://dt4l9bx31tioh.cloudfront.net/eazymedia/restaurant/684079/restaurant820220714065006.jpg",
  "rating": "4.0",
  "res_name": "Fuel Cafe",
  "location": "Banjara Hills, Hyderabad",
  "deal_text": "15% Off + 25% Off"
}


In [68]:
# ==============================================================
# EazyDiner Hyderabad Scraper (Confirmed Working Version)
# ==============================================================
# Requirements:
# pip install requests pandas
# ==============================================================

import requests
import pandas as pd

# JSON endpoint for Hyderabad
url = "https://www.eazydiner.com/_next/data/9Iz0uKJzU8-Mwrfvo8KeI/en/hyderabad.json"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*"
}

# Fetch JSON data
response = requests.get(url, headers=headers)
if response.status_code != 200:
    print(f"⚠️ Failed to fetch data (Status code: {response.status_code})")
    exit()

data = response.json()

# Navigate to restaurant list
restaurants = (
    data.get("pageProps", {})
        .get("data", {})
        .get("data", {})
        .get("buckets", {})
        .get("popular_restaurants", {})
        .get("items", [])
)

# Parse restaurant info
records = []
for rest in restaurants:
    name = rest.get("res_name")
    location = rest.get("location")
    rating = rest.get("rating")
    deal = rest.get("deal_text")
    img = rest.get("image")
    url_rest = (
        "https://www.eazydiner.com/" + rest.get("action_url", "").lstrip("/")
        if rest.get("action_url")
        else None
    )

    records.append({
        "Restaurant Name": name,
        "Location": location,
        "Rating": rating,
        "Deal": deal,
        "Image URL": img,
        "Restaurant URL": url_rest
    })

# Save results
df = pd.DataFrame(records)
print(df)
print(f"\n✅ Extracted {len(df)} restaurants.")
df.to_csv("eazydiner_hyderabad.csv", index=False, encoding="utf-8")
print("✅ Data saved to eazydiner_hyderabad.csv")


        Restaurant Name                               Location Rating  \
0             Fuel Cafe               Banjara Hills, Hyderabad    4.0   
1  Aster Cafe & Kitchen  Road No. 12, Banjara Hills, Hyderabad    4.0   
2             7 Sisters               Banjara Hills, Hyderabad    4.4   
3       La Pino'z Pizza  Road No. 12, Banjara Hills, Hyderabad    5.0   
4           Mazuri Cafe               Banjara Hills, Hyderabad    4.0   
5               Vasaara               Banjara Hills, Hyderabad    4.0   

                Deal                                          Image URL  \
0  15% Off + 25% Off  https://dt4l9bx31tioh.cloudfront.net/eazymedia...   
1  10% Off + 25% Off  https://dt4l9bx31tioh.cloudfront.net/eazymedia...   
2  30% Off + 25% Off  https://dt4l9bx31tioh.cloudfront.net/eazymedia...   
3  10% Off + 25% Off  https://dt4l9bx31tioh.cloudfront.net/eazymedia...   
4  30% Off + 25% Off  https://dt4l9bx31tioh.cloudfront.net/eazymedia...   
5  30% Off + 25% Off  https://dt4l9bx3

In [70]:
# ==============================================================
# EazyDiner Multi-City Restaurant Scraper (Fixed Version)
# ==============================================================
# Requirements:
# pip install requests pandas tqdm
# ==============================================================

import requests
import pandas as pd
from tqdm import tqdm
import time

BASE_URL = "https://www.eazydiner.com/_next/data/9Iz0uKJzU8-Mwrfvo8KeI/en"
CITIES_API = "https://force.eazydiner.com/web/all-cities"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*"
}

# --------------------------------------------------------------
# Step 1: Fetch all cities from EazyDiner
# --------------------------------------------------------------
resp = requests.get(CITIES_API, headers=headers)
if resp.status_code != 200:
    print("⚠️ Failed to fetch city list.")
    exit()

cities_data = resp.json().get("data", [])

# Handle both possible formats: list of dicts or list of strings
cities = []
for c in cities_data:
    if isinstance(c, dict):
        code = c.get("code")
        if code:
            cities.append(code)
    elif isinstance(c, str):
        cities.append(c.lower().strip())

print(f"✅ Found {len(cities)} cities on EazyDiner.\n")

# --------------------------------------------------------------
# Step 2: Loop through each city and collect restaurants
# --------------------------------------------------------------
all_restaurants = []

for city in tqdm(cities, desc="Scraping Cities"):
    city_url = f"{BASE_URL}/{city}.json"

    try:
        res = requests.get(city_url, headers=headers, timeout=20)
        if res.status_code != 200:
            print(f"⚠️ Skipping {city} (status {res.status_code})")
            continue

        data = res.json()
        restaurants = (
            data.get("pageProps", {})
                .get("data", {})
                .get("data", {})
                .get("buckets", {})
                .get("popular_restaurants", {})
                .get("items", [])
        )

        if not restaurants:
            continue

        for rest in restaurants:
            all_restaurants.append({
                "City": city,
                "Restaurant Name": rest.get("res_name"),
                "Location": rest.get("location"),
                "Rating": rest.get("rating"),
                "Deal": rest.get("deal_text"),
                "Image URL": rest.get("image"),
                "Restaurant URL": (
                    "https://www.eazydiner.com/" + rest.get("action_url", "").lstrip("/")
                    if rest.get("action_url") else None
                )
            })

        # Be polite — avoid overloading the server
        time.sleep(1)

    except Exception as e:
        print(f"❌ Error processing {city}: {e}")

# --------------------------------------------------------------
# Step 3: Save all data to CSV
# --------------------------------------------------------------
df = pd.DataFrame(all_restaurants)
print(f"\n✅ Extracted total {len(df)} restaurants across {len(cities)} cities.")

df.to_csv("eazydiner_all_cities.csv", index=False, encoding="utf-8")
print("✅ Data saved to eazydiner_all_cities.csv")

# Optional: Display sample output
print("\nSample:")
print(df.head())


✅ Found 3 cities on EazyDiner.



Scraping Cities:  33%|██████████████████████▎                                            | 1/3 [00:01<00:02,  1.47s/it]

⚠️ Skipping header (status 404)


Scraping Cities:  67%|████████████████████████████████████████████▋                      | 2/3 [00:03<00:01,  1.66s/it]

⚠️ Skipping popular_cities (status 404)


Scraping Cities: 100%|███████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.66s/it]

⚠️ Skipping unpopular_cities (status 404)

✅ Extracted total 0 restaurants across 3 cities.
✅ Data saved to eazydiner_all_cities.csv

Sample:
Empty DataFrame
Columns: []
Index: []


In [71]:
# ==============================================================
# EazyDiner Multi-City Scraper (Full City Extraction)
# ==============================================================
# Requirements:
# pip install requests pandas tqdm
# ==============================================================

import requests
import pandas as pd
from tqdm import tqdm
import time

BASE_URL = "https://www.eazydiner.com/_next/data/9Iz0uKJzU8-Mwrfvo8KeI/en"
CITIES_API = "https://force.eazydiner.com/web/all-cities"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*"
}

# --------------------------------------------------------------
# Step 1: Fetch all cities from EazyDiner
# --------------------------------------------------------------
resp = requests.get(CITIES_API, headers=headers)
if resp.status_code != 200:
    print("⚠️ Failed to fetch city list.")
    exit()

cities_raw = resp.json()

# Extract city codes properly
city_codes = []
for section in ["popular_cities", "unpopular_cities"]:
    section_data = cities_raw.get(section, {}).get("cities", [])
    for city in section_data:
        code = city.get("code")
        if code:
            city_codes.append(code.lower())

print(f"✅ Found {len(city_codes)} restaurant cities on EazyDiner.\n")

# --------------------------------------------------------------
# Step 2: Loop through each city and collect restaurants
# --------------------------------------------------------------
all_restaurants = []

for city in tqdm(city_codes, desc="Scraping Cities"):
    city_url = f"{BASE_URL}/{city}.json"

    try:
        res = requests.get(city_url, headers=headers, timeout=20)
        if res.status_code != 200:
            print(f"⚠️ Skipping {city} (status {res.status_code})")
            continue

        data = res.json()
        restaurants = (
            data.get("pageProps", {})
                .get("data", {})
                .get("data", {})
                .get("buckets", {})
                .get("popular_restaurants", {})
                .get("items", [])
        )

        if not restaurants:
            continue

        for rest in restaurants:
            all_restaurants.append({
                "City": city,
                "Restaurant Name": rest.get("res_name"),
                "Location": rest.get("location"),
                "Rating": rest.get("rating"),
                "Deal": rest.get("deal_text"),
                "Image URL": rest.get("image"),
                "Restaurant URL": (
                    "https://www.eazydiner.com/" + rest.get("action_url", "").lstrip("/")
                    if rest.get("action_url") else None
                )
            })

        time.sleep(1)

    except Exception as e:
        print(f"❌ Error processing {city}: {e}")

# --------------------------------------------------------------
# Step 3: Save all data to CSV
# --------------------------------------------------------------
df = pd.DataFrame(all_restaurants)
print(f"\n✅ Extracted total {len(df)} restaurants across {len(city_codes)} cities.")
df.to_csv("eazydiner_all_cities.csv", index=False, encoding="utf-8")
print("✅ Data saved to eazydiner_all_cities.csv")

print("\nSample:")
print(df.head())


✅ Found 0 restaurant cities on EazyDiner.



Scraping Cities: 0it [00:00, ?it/s]


✅ Extracted total 0 restaurants across 0 cities.
✅ Data saved to eazydiner_all_cities.csv

Sample:
Empty DataFrame
Columns: []
Index: []


In [73]:
import requests, json

url = "https://force.eazydiner.com/web/all-cities"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

r = requests.get(url, headers=headers)
data = r.json()

print("Top-level keys:", list(data.keys()))
print("\nSample (first 500 chars):\n")
print(json.dumps(data, indent=2)[:500])



Top-level keys: ['data', 'meta_title', 'meta_description']

Sample (first 500 chars):

{
  "data": {
    "header": "Book a Table in 338 cities in India and Dubai",
    "popular_cities": [
      {
        "url": "/delhi-ncr",
        "image": "https://dt4l9bx31tioh.cloudfront.net/eazymedia/cities/delhi-ncr-22.png",
        "name": "Delhi NCR",
        "code": "delhi-ncr",
        "restaurants_count": 17035
      },
      {
        "url": "/mumbai",
        "image": "https://dt4l9bx31tioh.cloudfront.net/eazymedia/cities/mumbai-22.png",
        "name": "Mumbai",
        "code": "mumb


In [74]:
# ==============================================================
# EazyDiner Multi-City Restaurant Scraper (Final Version)
# ==============================================================
# Requirements:
# pip install requests pandas tqdm
# ==============================================================

import requests
import pandas as pd
from tqdm import tqdm
import time

BASE_URL = "https://www.eazydiner.com/_next/data/9Iz0uKJzU8-Mwrfvo8KeI/en"
CITIES_API = "https://force.eazydiner.com/web/all-cities"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*"
}

# --------------------------------------------------------------
# Step 1: Fetch all city codes
# --------------------------------------------------------------
resp = requests.get(CITIES_API, headers=headers)
if resp.status_code != 200:
    print("⚠️ Failed to fetch city list.")
    exit()

data = resp.json()
cities_data = data.get("data", {}).get("popular_cities", [])
cities = [c.get("code") for c in cities_data if c.get("code")]

print(f"✅ Found {len(cities)} restaurant cities on EazyDiner.\n")

# --------------------------------------------------------------
# Step 2: Loop through cities and scrape restaurants
# --------------------------------------------------------------
all_restaurants = []

for city in tqdm(cities, desc="Scraping Cities"):
    city_url = f"{BASE_URL}/{city}.json"

    try:
        res = requests.get(city_url, headers=headers, timeout=20)
        if res.status_code != 200:
            print(f"⚠️ Skipping {city} (status {res.status_code})")
            continue

        data = res.json()
        restaurants = (
            data.get("pageProps", {})
                .get("data", {})
                .get("data", {})
                .get("buckets", {})
                .get("popular_restaurants", {})
                .get("items", [])
        )

        if not restaurants:
            continue

        for rest in restaurants:
            all_restaurants.append({
                "City": city,
                "Restaurant Name": rest.get("res_name"),
                "Location": rest.get("location"),
                "Rating": rest.get("rating"),
                "Deal": rest.get("deal_text"),
                "Image URL": rest.get("image"),
                "Restaurant URL": (
                    "https://www.eazydiner.com/" + rest.get("action_url", "").lstrip("/")
                    if rest.get("action_url") else None
                )
            })

        time.sleep(1)  # polite delay

    except Exception as e:
        print(f"❌ Error processing {city}: {e}")

# --------------------------------------------------------------
# Step 3: Save all data
# --------------------------------------------------------------
df = pd.DataFrame(all_restaurants)
print(f"\n✅ Extracted total {len(df)} restaurants across {len(cities)} cities.")
df.to_csv("eazydiner_all_cities.csv", index=False, encoding="utf-8")
print("✅ Data saved to eazydiner_all_cities.csv")

print("\nSample:")
print(df.head())


✅ Found 31 restaurant cities on EazyDiner.



Scraping Cities: 100%|█████████████████████████████████████████████████████████████████| 31/31 [01:31<00:00,  2.95s/it]


✅ Extracted total 184 restaurants across 31 cities.
✅ Data saved to eazydiner_all_cities.csv

Sample:
        City     Restaurant Name  \
0  delhi-ncr               Zing    
1  delhi-ncr             Fifty9    
2  delhi-ncr      Xero Courtyard   
3  delhi-ncr      Openhouse Cafe   
4  delhi-ncr  The Immigrant Cafe   

                                       Location Rating               Deal  \
0       The Metropolitan Hotel & Spa, New Delhi    1.0  25% Off + 25% Off   
1                Radisson Blu Marina, New Delhi    4.4  30% Off + 25% Off   
2  Janpath, Connaught Place (CP), Central Delhi    4.2  50% off + 25% Off   
3           Connaught Place (CP), Central Delhi    4.2  30% Off + 25% Off   
4           Connaught Place (CP), Central Delhi    4.3  30% Off + 25% Off   

                                           Image URL  \
0  https://dt4l9bx31tioh.cloudfront.net/eazymedia...   
1  https://dt4l9bx31tioh.cloudfront.net/eazymedia...   
2  https://dt4l9bx31tioh.cloudfront.net/eazymedia

In [75]:
# inside the city loop, before appending restaurants
page = 1
while True:
    page_suffix = f"/page-{page}.json" if page > 1 else ".json"
    city_url = f"{BASE_URL}/{city}{page_suffix}"

    res = requests.get(city_url, headers=headers, timeout=20)
    if res.status_code != 200:
        break

    data = res.json()
    items = (
        data.get("pageProps", {})
            .get("data", {})
            .get("data", {})
            .get("buckets", {})
            .get("popular_restaurants", {})
            .get("items", [])
    )

    if not items:
        break

    # same parsing loop here
    for rest in items:
        all_restaurants.append({...})

    page += 1
    time.sleep(1)


In [76]:
# ==============================================================
# EazyDiner Full India Scraper (with Pagination)
# ==============================================================
# Requirements:
# pip install requests pandas tqdm
# ==============================================================

import requests
import pandas as pd
from tqdm import tqdm
import time

BASE_URL = "https://www.eazydiner.com/_next/data/9Iz0uKJzU8-Mwrfvo8KeI/en"
CITIES_API = "https://force.eazydiner.com/web/all-cities"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*"
}

# --------------------------------------------------------------
# Step 1: Fetch all city codes
# --------------------------------------------------------------
resp = requests.get(CITIES_API, headers=headers)
if resp.status_code != 200:
    print("⚠️ Failed to fetch city list.")
    exit()

data = resp.json()
cities_data = data.get("data", {}).get("popular_cities", [])
cities = [c.get("code") for c in cities_data if c.get("code")]

print(f"✅ Found {len(cities)} restaurant cities on EazyDiner.\n")

# --------------------------------------------------------------
# Step 2: Scrape all pages for each city
# --------------------------------------------------------------
all_restaurants = []

for city in tqdm(cities, desc="Scraping Cities"):
    page = 1
    while True:
        # Build URL for each page
        page_suffix = f"/{city}/page-{page}.json" if page > 1 else f"/{city}.json"
        city_url = f"{BASE_URL}{page_suffix}"

        res = requests.get(city_url, headers=headers, timeout=20)
        if res.status_code != 200:
            break  # No more pages

        data = res.json()
        restaurants = (
            data.get("pageProps", {})
                .get("data", {})
                .get("data", {})
                .get("buckets", {})
                .get("popular_restaurants", {})
                .get("items", [])
        )

        if not restaurants:
            break  # Stop if no restaurants found on this page

        for rest in restaurants:
            all_restaurants.append({
                "City": city,
                "Restaurant Name": rest.get("res_name"),
                "Location": rest.get("location"),
                "Rating": rest.get("rating"),
                "Deal": rest.get("deal_text"),
                "Image URL": rest.get("image"),
                "Restaurant URL": (
                    "https://www.eazydiner.com/" + rest.get("action_url", "").lstrip("/")
                    if rest.get("action_url") else None
                )
            })

        print(f"✅ {city} - Page {page} ({len(restaurants)} restaurants)")
        page += 1
        time.sleep(1)

# --------------------------------------------------------------
# Step 3: Save to CSV
# --------------------------------------------------------------
df = pd.DataFrame(all_restaurants)
print(f"\n✅ Extracted total {len(df)} restaurants across {len(cities)} cities.")
df.to_csv("eazydiner_all_cities_full.csv", index=False, encoding="utf-8")
print("✅ Data saved to eazydiner_all_cities_full.csv")

print("\nSample:")
print(df.head())


✅ Found 31 restaurant cities on EazyDiner.



Scraping Cities:   0%|                                                                          | 0/31 [00:00<?, ?it/s]

✅ delhi-ncr - Page 1 (6 restaurants)


Scraping Cities:   3%|██▏                                                               | 1/31 [00:02<01:00,  2.03s/it]

✅ mumbai - Page 1 (6 restaurants)


...

✅ chandigarh-tricity - Page 1 (6 restaurants)


Scraping Cities:  97%|██████████████████████████████████████████████████████████████▉  | 30/31 [02:13<00:07,  7.21s/it]

✅ mangalore-tricity - Page 1 (6 restaurants)


Scraping Cities: 100%|█████████████████████████████████████████████████████████████████| 31/31 [02:16<00:00,  4.40s/it]


✅ Extracted total 184 restaurants across 31 cities.
✅ Data saved to eazydiner_all_cities_full.csv

Sample:
        City     Restaurant Name  \
0  delhi-ncr               Zing    
1  delhi-ncr             Fifty9    
2  delhi-ncr      Xero Courtyard   
3  delhi-ncr      Openhouse Cafe   
4  delhi-ncr  The Immigrant Cafe   

                                       Location Rating               Deal  \
0       The Metropolitan Hotel & Spa, New Delhi    1.0  25% Off + 25% Off   
1                Radisson Blu Marina, New Delhi    4.4  30% Off + 25% Off   
2  Janpath, Connaught Place (CP), Central Delhi    4.2  50% off + 25% Off   
3           Connaught Place (CP), Central Delhi    4.2  30% Off + 25% Off   
4           Connaught Place (CP), Central Delhi    4.3  30% Off + 25% Off   

                                           Image URL  \
0  https://dt4l9bx31tioh.cloudfront.net/eazymedia...   
1  https://dt4l9bx31tioh.cloudfront.net/eazymedia...   
2  https://dt4l9bx31tioh.cloudfront.net/eazy